In [ ]:
!pip install flask-ngrok

In [ ]:
import pandas as pd
import pickle
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import string
import re
import nltk
from keras.models import load_model,Model
# load in all the modules we're going to need
import nltk,re,string,collections
import tensorflow as tf
from flask import Flask,request,jsonify,render_template,abort,make_response 
from flask_ngrok import run_with_ngrok
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence, text
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive//My Drive/Tweet Sentiment Extraction/updated_train.pkl','rb') as f:
     train=pickle.load(f)  

In [ ]:
text_split=train['text'].apply(lambda x: len(str(x).split())).tolist()
y=np.zeros((train.shape[0],max(text_split)+1))
for i in range(train.shape[0]):
  start=train['start_index'][i]
  end=train['end_index'][i]
  y[i][start:end+1]=1
X=train[['textID','text','selected_text','sentiment']]
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.15,random_state=42) 

In [ ]:
app = Flask(__name__,template_folder='/content/drive//My Drive/Tweet Sentiment Extraction/templates/')
run_with_ngrok(app)

In [ ]:
#https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
def evaluate(tweet,sentiment):
  tweet=str(tweet)
  sentiment=str(sentiment)
  tweet=tweet.lower()
  #storing the puntuation free text
  tweet=re.sub('https?://\S+|www\.\S+', '', tweet)
  #https://stackoverflow.com/questions/12851791/removing-numbers-from-string
  #https://stackoverflow.com/questions/18082130/python-regex-to-remove-all-words-which-contains-number
  tweet=re.sub('\S*\d\S*',' ',tweet) #Removing Numbers
  tweet=re.sub('<.*?>+',' ',tweet)   #Removing Angular Brackets
  tweet=re.sub('\[.*?\]',' ',tweet)  #Removing Square Brackets
  tweet=re.sub('\n',' ',tweet)       #Removing '\n' character 
  tweet=re.sub('\*+','<ABUSE>',tweet) #Replacing **** by ABUSE word
  #https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/
  tweet="".join([i for i in tweet if i not in string.punctuation])
  
  print(tweet,sentiment)

  train_text=X_train['text'].values
  train_sentiment=X_train['sentiment'].values

  
  #Reference : https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

  tokenizer1=Tokenizer(lower=True,split=' ',filters='!"#$%&()*+,-./:;<=>?@[\\]^_{|}~\t\n',oov_token='<unw>')
  tokenizer1.fit_on_texts(train_text)
  max_len_text=32
  train_text=tokenizer1.texts_to_sequences(train_text)
  train_text=sequence.pad_sequences(train_text,maxlen=max_len_text,padding='post')
  word_index_text=tokenizer1.word_index

  
  tokenizer2=Tokenizer(lower=True,split=' ',filters='!"#$%&()*+,-./:;<=>?@[\\]^_{|}~\t\n',oov_token='<unw>')
  tokenizer2.fit_on_texts(train_sentiment)
  max_len_sentiment=1
  train_sentiment=tokenizer2.texts_to_sequences(train_sentiment)
  train_sentiment=sequence.pad_sequences(train_sentiment,maxlen=max_len_sentiment,padding='post')
  word_index_sentiment=tokenizer2.word_index

  
  model=load_model('/content/drive//My Drive/Tweet Sentiment Extraction/Bi-LSTM.h5')
 
  text=[]
  for k in tweet.split():
    if k in word_index_text:
      text.append(word_index_text[k])
    else:
      text.append(0)
  while len(text)<32:
    text.append(0)
  text=np.reshape(text,(1,32))
    
  s=1
  if sentiment in word_index_sentiment:
    s=word_index_sentiment[sentiment]
    s=np.reshape(s,(1,1))

  tweet_pred=model.predict([text,s])
  tweet_pred=np.squeeze(tweet_pred)
  tweet_pred=np.round(tweet_pred)
  tweet_pred=np.reshape(tweet_pred,(1,33))
  print(tweet_pred)

  pred=[]
  for vector in tweet_pred:
    index=[]
    for i,value in enumerate(vector):
      if value == 1:
        index.append(i)
    pred.append(index)
  pred=pred[0]
  print(pred)


  def get_text(tweet,index):
    predict=[]
    tweet=tweet.split()
    l=len(tweet)
    for i in index:
      if i < l:
        predict.append(tweet[i])
    return predict

  pred_tweet=get_text(tweet,pred)
  pred_tweet=" ".join([i for i in pred_tweet])
  return pred_tweet


In [ ]:
@app.route('/')
def home():
    return render_template('home.html')    

@app.route('/predict',methods=["GET","POST"])
def predict():
    if request.method == "POST":
        # getting input with name = tweet in HTML form
        t=request.form.get("tweet")
        # getting input with name = sentiment in HTML form 
        s=request.form.get("sentiment") 
        x=evaluate(t,s)
        return render_template('extraction.html',e=x,tweet=t,sentiment=s) 
            
if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c0e1-34-122-65-239.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Oct/2021 11:10:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 11:10:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Oct/2021 11:10:49] "GET / HTTP/1.1" 200 -


love is something like the clouds that were in the sky before the sun came out she replied positive


[2021-10-07 11:11:31,870] ERROR in app: Exception on /predict [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2098, in make_response
    "The view function did not return a valid response. The"
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [07/Oct/2021 11:11:31] "GET /predict HTTP/1.1" 500 -


[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 0.]]
[0, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


127.0.0.1 - - [07/Oct/2021 11:11:34] "POST /predict HTTP/1.1" 200 -
